## TODO
~~1. Batching!~~   
    ~~-Encoder.Forward의 input 모양 어떻게 되지? / .view 인자 확인!~~
2. Attention  
3. Teacher Forcing  
~~4. Parameter(things to be updated) 등록 잘 됐나 확인(= 그래프 잘 그린 건지 어케 확인하나?)~~
5. Train / Dev 사전에 나누기
6. Pretrained word vector을 쓸 수가 있나..?  
** SOS, EOS, PAD token 관리 어디서?**   
** INTO GPU: x batch, y batch, parameter On GPU**
7. model save등등 train 뒷단 얘기  
    -torch.save


#### NOTE
1. Decoder가 2 layer일때, initial hidden?  
    - https://discuss.pytorch.org/t/understanding-output-of-lstm/12320/2
    - hidden 의 dimension
2. embedding 거친 후엔 batch_size x seq_len x embedding_dim 임!
    - embedding input은 batch_size x seq_len
2. LSTM의 batch_first
3. LSTM input의 dimension
4. NLLLoss()의 dimension

In [150]:
from build_vocab import *

In [151]:
meta = build_meta()
tagVocab, rvVocab = filter_by_cnt(2, 2, 'data/products.pkl')
data = prepareData(meta, tagVocab, rvVocab, 'data/products.pkl')

Building vocab for tags ...
Building vocab for reviews ...
3339 out of 3465 words left, which is 96.36363636363636 %
21036 out of 37012 words left, which is 56.83562087971469 %


In [152]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from tqdm import tqdm_notebook

In [153]:
class Config():
    dropout = 0.2
    batch_size = 2
    num_steps = 10
    
    # Encoder
    rating_size = len(meta.rating2idx)
    category_size = len(meta.lowcat2idx)
    tag_size = len(tagVocab.word2idx)    # tag vocab_size
    pretrained = False 
    attribute_size = 64
    hidden_size = 512 # fixed-vector size 
    
    # Decoder
    num_layers = 2
    output_size =  len(rvVocab.word2idx)    # review vocab_size
    
    # TEMP - 이거 어디서 관리하지? 여긴 아닌뎅
    padding_idx = 0

In [154]:
class Encoder(nn.Module):
    def __init__(self, config):        
        super().__init__()
        self.config = config
        
        self.emb_rating = nn.Embedding(self.config.rating_size, self.config.attribute_size)   
        self.emb_category = nn.Embedding(self.config.category_size, self.config.attribute_size)
        self.emb_tag = nn.Embedding(self.config.tag_size, self.config.attribute_size,
                                   padding_idx=self.config.padding_idx)        
        self.out = nn.Linear(self.config.attribute_size * 3, self.config.hidden_size*self.config.num_layers)
        self.init_hidden()
        
    def forward(self, rating, category, tag):
        """
        Inputs:
            rating: TENSOR of shape (batch_size, 1)
            category: TENSOR of shape (batch_size, 1)
            tag : 1) TENSOR of shape (batch_size, tag_MAXLEN)
        Returns:
            concatenated attr for attention, encoder_output
        """
        
        assert len(rating) == len(category) == len(tag)
        attr_rating = self.emb_rating(rating)    # view no need!
        attr_category = self.emb_category(category)
        tag_len = self.get_tag_len(tag)    
        attr_tag = torch.sum(self.emb_tag(tag), 1, keepdim=True) / tag_len    # CBOW
                # 여기서 이렇게 해도 backprop 맞게 되나??
        
        attr = torch.cat((attr_rating, attr_category, attr_tag), 2)
        out = self.out(attr)
        encoder_output = F.tanh(out)
        return attr, encoder_output
    
    def get_tag_len(self, tag): 
        """padding 제외한 token 개수"""
        return torch.sum(tag!=self.config.padding_idx, 1).unsqueeze(1).unsqueeze(1).type(torch.float)
        
    def init_hidden(self):
        for param in self.parameters():
            nn.init.uniform_(param, -0.08, 0.08)

In [155]:
print("===== Testing encoder... with single input =====")
config = Config()
encoder = Encoder(config)
rating = torch.tensor([[3]]).type(torch.long)    # idx of rating in tensor
category = torch.tensor([[7]]).type(torch.long)  # idx of category in tensor
tag = torch.tensor([[1,2,1]]).type(torch.long)    # idxs of tag
attr, encoder_output = encoder(rating,category,tag)
print(attr.size())
print(encoder_output.size())

print("\n===== with multiple inputs =====")
rating = torch.tensor([[3],[4]])
category = torch.tensor([[8],[1]])
tag = torch.tensor([[1,2,1,2], [0,1,1,0]])    # 이렇게 하려면 padding 되어 있어야해!
attr, encoder_output = encoder(rating,category,tag)
print(attr.size())
print(encoder_output.size())

===== Testing encoder... with single input =====
torch.Size([1, 1, 192])
torch.Size([1, 1, 1024])

===== with multiple inputs =====
torch.Size([2, 1, 192])
torch.Size([2, 1, 1024])


In [156]:
class Decoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        # TODO: if self.config.pretrained = True
        self.embedding = nn.Embedding(self.config.output_size, self.config.hidden_size)
        self.lstm = nn.LSTM(self.config.hidden_size, self.config.hidden_size, \
                            num_layers=self.config.num_layers, dropout=self.config.dropout, \
                           batch_first=True)
        self.out = nn.Linear(self.config.hidden_size, self.config.output_size)
        
    def forward(self, input_token, hidden):
        """
        Inputs:
            input_token: TENSOR of shape (batch_size, 1)
            hidden: from last hidden of encoder
        Returns:
        """
        # 가운데 1이니까 unroll방식으로만!
        output = self.embedding(input_token)
        # LSTM의 hidden은 (hx, cx)
        output, hidden = self.lstm(output, hidden)
        output = self.out(output)
        output = F.log_softmax(output, dim=2)
        return output, hidden

    def initHidden(self):
        pass

In [157]:
print("====testing decoder with encoder_output...(batch=2)====\n")
decoder = Decoder(config)
# h_0을 초기화시키고, c_0은 냅두기!!!(0으로 하든가 임의값으로 하든가)
batch_size = encoder_output.size(0)    # same as len(encoder_output) and 
h_ = encoder_output.view(config.num_layers, 2, config.hidden_size)    # 2는 batch_size
c_ = torch.zeros_like(h_)
hidden = (h_, c_)

input_token = torch.tensor([[1],[2]])  # shape (batch_size, seq_len)
output, hidden = decoder(input_token, hidden)
print("input_token.size(): ", input_token.size())
print("hidden[0].size(): ", hidden[0].size())
print("hidden[1].size(): ", hidden[1].size())
print("output.size(): ", output.size())



====testing decoder with encoder_output...(batch=2)====

input_token.size():  torch.Size([2, 1])
hidden[0].size():  torch.Size([2, 2, 512])
hidden[1].size():  torch.Size([2, 2, 512])
output.size():  torch.Size([2, 1, 21038])


In [162]:
def splitHidden(encoder_output, config):
    h_0 = encoder_output.view(config.num_layers, encoder_output.size(0), \
                              config.hidden_size)
    c_0 = torch.zeros_like(h_0) 
    return (h_0, c_0)

def train(encoder, decoder, data, loss_fn, optimizer, batch_size, num_steps):
    encoder.train()
    decoder.train()

    for t in tqdm_notebook(range(num_steps)):
        optimizer.zero_grad()
        rating_tensor, category_tensor, tag_tensor, target_tensor = data_iterator(data, batch_size)
        target_length = target_tensor.size(-1)
        
        attr, encoder_output = encoder(rating_tensor, category_tensor, tag_tensor)
        decoder_hidden = splitHidden(encoder_output, encoder.config)        
        
        decoder_input = torch.zeros((batch_size,1)).type(torch.long)    # SOS token
        loss = 0
        #print("target_length in this batch: %d" % target_length)
        for idx in range(target_length): 
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)            
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.detach().view(batch_size, 1)
            loss += loss_fn(decoder_output.squeeze(), target_tensor[:,idx])
        num_actual_token = torch.sum(target_tensor != encoder.config.padding_idx).item()
        loss /= num_actual_token
        print("loss: %f" % loss)
        
        loss.backward()
#         nn.utils.clip_grad_norm_(encoder.parameters(), 0.25)
#         nn.utils.clip_grad_norm_(encoder.parameters(), 0.25)
        optimizer.step()     

#         for name, param in decoder.named_parameters():
#             print("decoder "+name, param.grad)
#         for name, param in encoder.named_parameters():
#             print("encoder "+name, param.grad)    


In [167]:
config = Config()
encoder = Encoder(config)
decoder = Decoder(config)
# data is defined on top
params = list(encoder.parameters()) + list(decoder.parameters())
loss_fn = nn.NLLLoss(size_average=False, ignore_index=config.padding_idx)
optimizer = optim.Adam(params, lr=0.002)
#train(encoder, decoder, data, loss_fn, optimizer, config.batch_size, config.num_steps)
train(encoder, decoder, data, loss_fn, optimizer, batch_size=30, num_steps=1000)

Exception in thread Thread-89:
Traceback (most recent call last):
  File "/home/nlp908/anaconda3/envs/hwijeen_3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/nlp908/anaconda3/envs/hwijeen_3.6/lib/python3.6/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/home/nlp908/anaconda3/envs/hwijeen_3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



loss: 9.958877
loss: 9.913923
loss: 9.790391
loss: 9.021402
loss: 8.166480
loss: 7.414064
loss: 7.272287
loss: 6.905063
loss: 6.793014
loss: 6.947257
loss: 7.099370
loss: 6.957481
loss: 6.647564
loss: 6.983301
loss: 7.597042
loss: 7.219642
loss: 6.935116
loss: 7.469126
loss: 6.923403
loss: 7.166224
loss: 6.977667
loss: 7.349732
loss: 7.071460
loss: 6.831165
loss: 6.951372
loss: 6.689351
loss: 6.972092
loss: 6.986296
loss: 6.891676
loss: 6.695094
loss: 7.070732
loss: 7.055617
loss: 7.232532
loss: 6.935348
loss: 7.107073
loss: 6.647736
loss: 6.915483
loss: 6.622308
loss: 7.273326
loss: 7.156304
loss: 6.969756
loss: 6.842958
loss: 6.795801
loss: 6.873309
loss: 6.774458
loss: 6.709067
loss: 6.811054
loss: 6.624053
loss: 6.917876
loss: 6.663665
loss: 6.901025
loss: 6.628020
loss: 6.941717
loss: 6.611979
loss: 6.421379
loss: 6.946917
loss: 7.183492
loss: 6.769335
loss: 6.534405
loss: 6.901868
loss: 6.846897
loss: 6.745937
loss: 6.702459
loss: 6.395546
loss: 6.509654
loss: 6.792053
loss: 6.57

loss: 6.552084
loss: 5.805659
loss: 6.380638
loss: 6.515791
loss: 6.319370
loss: 6.559206
loss: 6.476161
loss: 6.117534
loss: 6.170471
loss: 6.270523
loss: 6.398271
loss: 6.335836
loss: 6.407510
loss: 6.468328
loss: 6.322253
loss: 6.627347
loss: 6.447699
loss: 6.006077
loss: 6.163225
loss: 6.569389
loss: 6.571123
loss: 6.350945
loss: 6.225169
loss: 6.168538
loss: 6.491021
loss: 6.069513
loss: 6.352233
loss: 5.965123
loss: 6.042046
loss: 6.468031
loss: 6.096445
loss: 6.130833
loss: 6.352761
loss: 6.418515
loss: 6.353198
loss: 6.394598
loss: 6.425023
loss: 6.503269
loss: 6.076306
loss: 6.265134
loss: 6.436759
loss: 5.957347
loss: 6.480291
loss: 6.475164
loss: 6.333910
loss: 6.826436
loss: 6.085797
loss: 6.519462
loss: 6.667211
loss: 6.477669
loss: 5.848251
loss: 6.280698
loss: 6.527968
loss: 6.408465
loss: 6.586850
loss: 6.279148
loss: 6.368783
loss: 6.185057
loss: 6.586433
loss: 5.599621
loss: 6.236046
loss: 6.170139
loss: 6.609645
loss: 6.417264
loss: 6.403788
loss: 6.373801
loss: 6.32

KeyboardInterrupt: 

In [16]:
class Attr2Seq(nn.Module):
    def __init__(self, config, criterion):
        super().__init__()
        self.config = config
        self.criterion = criterion
        self.encoder = Encoder(config)
        self.decoder = Decoder(config)
    
    def forward(self, rating, category, tag, target_tensor):
        # 함수 호출시 *[rating, category, tag]하기!

        batch_size = target_tensor.size(0)
        target_length = target_tensor.size(-1)
        attr, encoder_output = self.encoder(rating,category,tag)
        batch_size = encoder_output.size(0)
        decoder_hidden = self.splitHidden(encoder_output)
        input_token = torch.zeros((batch_size,1)).type(torch.long)    # SOS token
        
        decoder_outputs = []
        for idx in range(target_length): 
            decoder_output, decoder_hidden = decoder(input_token, decoder_hidden)            
            topv, topi = decoder_output.topk(1)
            input_token = topi.detach().view(batch_size, 1)
            decoder_outputs.append(decoder_output)

        # 이 아래 두 개 shape 조정은 loss function 요구 사항 맞추기 위함!
        decoder_outputs = torch.cat(decoder_outputs, 1).view(batch_size*target_length, -1)
        target_tensor = target_tensor.view(-1)
        loss = self.criterion(decoder_outputs, target_tensor) #/ batch_size
                        # batch_size로 나눠주기
#         print("decoder_outputs: ", decoder_outputs)
#         print("target_tensor: ", target_tensor)
        return loss
    
    def splitHidden(self, encoder_output):
        h_0 = encoder_output.view(self.config.num_layers, encoder_output.size(0), \
                                  self.config.hidden_size)
        c_0 = torch.zeros_like(h_0) 
        return (h_0, c_0)
    
    def inference(self):
        pass


In [17]:
print("==== testing Attr2Seq model... ====")
criterion = nn.NLLLoss()     # TODO: padding loss빼주기ㅠㅠ
model = Attr2Seq(config, criterion)
input_list = [torch.tensor([[2]]).type(torch.long), torch.tensor([[7]]).type(torch.long),
              torch.tensor([[0,2,1]])]
target_tensor = torch.tensor([[4]])
loss = model(*input_list, target_tensor)
print(loss)

print("\n==== baching(size 2) ====")
input_list = [torch.tensor([[2],[1]]).type(torch.long), torch.tensor([[7],[5]]).type(torch.long),
              torch.tensor([[0,2,1],[2,0,1]])]
target_tensor = torch.tensor([[9,2,1,3,5], [3,2,3,2,1]])
loss = model(*input_list, target_tensor)
print(loss)

==== testing Attr2Seq model... ====
tensor(10.0135)

==== baching(size 2) ====
tensor(10.0085)


In [54]:
# 그 전에 Train / Dev 나누기
# TODO: data랑 받아서 정의된 data iterator로 불러오기

def train(data, model, optimizer, num_steps):
    model.train()

    for t in tqdm_notebook(range(num_steps)):
#        a = list(model.parameters())[0]
        optimizer.zero_grad()
        r, c, t, rv = data_iterator(data, batch_size = 2)
        loss = model(r, c, t, rv)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()        
#         for name, param in model.named_parameters():
#             print(name)
#             print(param.grad)
#        b = list(model.parameters())[0]
#        print(loss)
    return loss.item()# / target_length

In [55]:
model = Attr2Seq(config, criterion)
optimizer = optim.SGD(model.parameters(), lr=1)
#optimizer = optim.RMSprop(model.parameters(), lr=0.2, alpha=0.95)
temp_loss = train(Encoded, model, optimizer, 2)
print(temp_loss)


nan


In [101]:
torch.cuda.device_count()

2

In [104]:
torch.cuda.is_available()

True

Building vocab for tags ...
Building vocab for reviews ...
3339 out of 3465 words left, which is 96.36363636363636 %
22193 out of 39173 words left, which is 56.653817680545274 %


In [56]:
class Attr2Seq(nn.Module):
    def __init__(self, config, criterion):
        super().__init__()
        self.config = config
        self.criterion = criterion
        self.encoder = Encoder(config)
        self.decoder = Decoder(config)
    
    def forward(self, rating, category, tag, target_tensor):

        batch_size = target_tensor.size(0)
        target_length = target_tensor.size(-1)
        attr, encoder_output = self.encoder(rating,category,tag)
        batch_size = encoder_output.size(0)
        decoder_hidden = self.splitHidden(encoder_output)
        input_token = torch.zeros((batch_size,1)).type(torch.long)    # SOS token
        
        loss = 0
        for idx in range(target_length): 
            decoder_output, decoder_hidden = decoder(input_token, decoder_hidden)            
            topv, topi = decoder_output.topk(1)
            input_token = topi.detach().view(batch_size, 1)
            loss += self.criterion(decoder_output.squeeze(), target_tensor[:,idx].view(-1))

        loss = loss / batch_size
        return loss
    
    def splitHidden(self, encoder_output):
        h_0 = encoder_output.view(self.config.num_layers, encoder_output.size(0), \
                                  self.config.hidden_size)
        c_0 = torch.zeros_like(h_0) 
        return (h_0, c_0)
    
    def inference(self):
        pass


In [60]:
# 그 전에 Train / Dev 나누기
# TODO: data랑 받아서 정의된 data iterator로 불러오기

def train(data, model, criterion, optimizer, num_steps):
    model.train()
    loss_fn = criterion
    for t in tqdm_notebook(range(num_steps)):
        a = list(model.parameters())[0]
        optimizer.zero_grad()
        r, c, t, rv = data_iterator(data, batch_size = 2)
            
        loss = model(r, c, t, rv)
#        decoder_output = model(r, c, t, rv)
#        rv = rv.view(-1)
#        loss = loss_fn(decoder_output, rv)
        loss.backward()
#        print(loss.grad_fn)
        print(loss)
        
        #nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()        
        for name, param in model.named_parameters():
            print(name)
            print(param.grad)
        b = list(model.parameters())[0]
        
        print(loss)
    return loss.item()# / target_length

In [61]:
criterion = nn.NLLLoss()
model = Attr2Seq(config, criterion)
params = list(model.encoder.parameters()) + list(model.decoder.parameters())
#optimizer = optim.SGD(model.parameters(), lr=1)
optimizer = optim.SGD(params, lr=1)

#optimizer = optim.RMSprop(model.parameters(), lr=0.2, alpha=0.95)
temp_loss = train(Encoded, model, criterion, optimizer, 10)
print(temp_loss)

Exception in thread Thread-22:
Traceback (most recent call last):
  File "/home/nlp908/anaconda3/envs/hwijeen_3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/nlp908/anaconda3/envs/hwijeen_3.6/lib/python3.6/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/home/nlp908/anaconda3/envs/hwijeen_3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



tensor(239.8178)
encoder.emb_rating.weight
tensor(1.00000e-03 *
       [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,

tensor(55.0357)
encoder.emb_rating.weight
tensor(1.00000e-03 *
       [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 

tensor([[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        ...,
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.]])
encoder.out.weight
tensor([[nan., nan., nan.,  ..., nan., nan., nan.],
        [nan., nan., nan.,  ..., nan., nan., nan.],
        [nan., nan., nan.,  ..., nan., nan., nan.],
        ...,
        [nan., nan., nan.,  ..., nan., nan., nan.],
        [nan., nan., nan.,  ..., nan., nan., nan.],
        [nan., nan., nan.,  ..., nan., nan., nan.]])
encoder.out.bias
tensor([nan., nan., nan.,  ..., nan., nan., nan.])
decoder.embedding.weight
None
decoder.lstm.weight_ih_l0
None
decoder.lstm.weight_hh_l0
None
decoder.lstm.bias_ih_l0
None
decoder.lstm.bias_hh_l0
None
decoder.lstm.weight_ih_l1
None
decoder.lstm.weight_hh_l1
None
decoder.lstm.bias_ih_l1
None
decoder.lstm.bias_hh

tensor([[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [nan., nan., nan.,  ..., nan., nan., nan.],
        ...,
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.]])
encoder.out.weight



KeyboardInterrupt: 